In [ ]:


from shapely.geometry import Point
from torch.utils.data import Dataset

class ShapelyFeatureDataset(Dataset):

  def __init__(self, features, transform=None):
    self.features = features
    self.transform = transform

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    feature = self.features[idx]

    if self.transform:
      feature = self.transform(feature)

    raster = self.feature_to_raster(feature)
    return raster

  def feature_to_raster(self, feature):
    # Create a blank raster
    raster = np.zeros((256, 256), dtype=np.uint8)
    # Convert feature to raster
    coords = np.array(feature.exterior.coords)
    cv2.fillPoly(raster, [coords.astype(np.int32)], 255)
    return raster

class RandomTransform:

  def __init__(self, scale_range=(0.5, 1.5), rotation_range=(-45, 45), noise_level=0.1):
    self.scale_range = scale_range
    self.rotation_range = rotation_range
    self.noise_level = noise_level

  def __call__(self, feature):
    # Random scale
    scale_factor = random.uniform(*self.scale_range)
    feature = scale(feature, xfact=scale_factor, yfact=scale_factor, origin='center')

    # Random rotation
    angle = random.uniform(*self.rotation_range)
    feature = rotate(feature, angle, origin='center')

    # Add noise
    noise = np.random.normal(0, self.noise_level, feature.exterior.coords.shape)
    noisy_coords = feature.exterior.coords + noise
    feature = shape({'type': 'Polygon', 'coordinates': [noisy_coords]})

    return feature

# Example usage
features = [Point(128, 128).buffer(50)]  # Example Shapely features
transform = RandomTransform()
dataset = ShapelyFeatureDataset(features, transform=transform)

# Get a sample
sample = dataset[0]
print(sample)

In [ ]:

from shapely.geometry import Point
from torch.utils.data import Dataset

class ShapelyFeatureDataset(Dataset):

  def __init__(self, features, transform=None, subset_size=5):
    self.features = features
    self.transform = transform
    self.subset_size = subset_size

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    subset = self.sample_subset()
    rasters, masks = self.features_to_rasters(subset)
    return rasters, masks

  def sample_subset(self):
    return random.sample(self.features, self.subset_size)

  def features_to_rasters(self, features):
    rasters = []
    masks = []
    placed_features = []
    for feature in features:
      if self.transform:
        feature = self.transform(feature)
      feature = self.ensure_no_overlap(feature, placed_features)
      raster, mask = self.feature_to_raster(feature)
      rasters.append(raster)
      masks.append(mask)
      placed_features.append(feature)
    return rasters, masks

  def ensure_no_overlap(self, feature, placed_features):
    minx, miny, maxx, maxy = feature.bounds
    width, height = maxx - minx, maxy - miny
    while True:
      x_offset = random.randint(0, 256 - int(width))
      y_offset = random.randint(0, 256 - int(height))
      translated_feature = translate(feature, xoff=x_offset - minx, yoff=y_offset - miny)
      if not any(translated_feature.intersects(pf) for pf in placed_features):
        return translated_feature

  def feature_to_raster(self, feature):
    # Create a blank raster and mask
    raster = np.zeros((256, 256), dtype=np.uint8)
    mask = np.zeros((256, 256), dtype=np.uint8)

    # Convert feature to raster
    coords = np.array(feature.exterior.coords)
    cv2.fillPoly(raster, [coords.astype(np.int32)], 255)
    cv2.fillPoly(mask, [coords.astype(np.int32)], 1)
    return raster, mask

class RandomTransform:

  def __init__(self, scale_range=(0.5, 1.5), rotation_range=(-45, 45), noise_level=0.1):
    self.scale_range = scale_range
    self.rotation_range = rotation_range
    self.noise_level = noise_level

  def __call__(self, feature):
    # Random scale
    scale_factor = random.uniform(*self.scale_range)
    feature = scale(feature, xfact=scale_factor, yfact=scale_factor, origin='center')

    # Random rotation
    angle = random.uniform(*self.rotation_range)
    feature = rotate(feature, angle, origin='center')

    # Add noise
    noise = np.random.normal(0, self.noise_level, feature.exterior.coords.shape)
    noisy_coords = feature.exterior.coords + noise
    feature = shape({'type': 'Polygon', 'coordinates': [noisy_coords]})

    return feature

# Example usage
features = [Point(128, 128).buffer(50) for _ in range(10)]  # Example Shapely features
transform = RandomTransform()
dataset = ShapelyFeatureDataset(features, transform=transform, subset_size=5)

# Get a sample
rasters, masks = dataset[0]
for i, (raster, mask) in enumerate(zip(rasters, masks)):
  print(f"Raster {i}:\n{raster}\nMask {i}:\n{mask}")


In [ ]:
import random
import cv2
import numpy as np
from shapely.affinity import rotate, scale, translate
from shapely.geometry import Point, shape
from torch.utils.data import Dataset

class ShapelyFeatureDataset(Dataset):

  def __init__(self, categories, transform=None, subset_size=5):
    self.categories = categories
    self.transform = transform
    self.subset_size = subset_size

  def __len__(self):
    return sum(len(features) for features in self.categories.values())

  def __getitem__(self, idx):
    subset = self.sample_subset()
    rasters, masks = self.features_to_rasters(subset)
    return rasters, masks

  def sample_subset(self):
    subset = []
    for category, features in self.categories.items():
      subset.extend(random.sample(features, min(self.subset_size, len(features))))
    return subset

  def features_to_rasters(self, features):
    rasters = []
    masks = []
    placed_features = []
    for feature in features:
      if self.transform:
        feature = self.transform(feature)
      feature = self.ensure_no_overlap(feature, placed_features)
      raster, mask = self.feature_to_raster(feature)
      rasters.append(raster)
      masks.append(mask)
      placed_features.append(feature)
    return rasters, masks

  def ensure_no_overlap(self, feature, placed_features):
    minx, miny, maxx, maxy = feature.bounds
    width, height = maxx - minx, maxy - miny
    while True:
      x_offset = random.randint(0, 256 - int(width))
      y_offset = random.randint(0, 256 - int(height))
      translated_feature = translate(feature, xoff=x_offset - minx, yoff=y_offset - miny)
      if not any(translated_feature.intersects(pf) for pf in placed_features):
        return translated_feature

  def feature_to_raster(self, feature):
    # Create a blank raster and mask
    raster = np.zeros((256, 256), dtype=np.uint8)
    mask = np.zeros((256, 256), dtype=np.uint8)

    # Convert feature to raster
    coords = np.array(feature.exterior.coords)
    cv2.fillPoly(raster, [coords.astype(np.int32)], 255)
    cv2.fillPoly(mask, [coords.astype(np.int32)], 1)
    return raster, mask

class RandomTransform:

  def __init__(self, scale_range=(0.5, 1.5), rotation_range=(-45, 45), noise_level=0.1):
    self.scale_range = scale_range
    self.rotation_range = rotation_range
    self.noise_level = noise_level

  def __call__(self, feature):
    # Random scale
    scale_factor = random.uniform(*self.scale_range)
    feature = scale(feature, xfact=scale_factor, yfact=scale_factor, origin='center')

    # Random rotation
    angle = random.uniform(*self.rotation_range)
    feature = rotate(feature, angle, origin='center')

    # Add noise
    noise = np.random.normal(0, self.noise_level, feature.exterior.coords.shape)
    noisy_coords = feature.exterior.coords + noise
    feature = shape({'type': 'Polygon', 'coordinates': [noisy_coords]})

    return feature

# Example usage
categories = {
    'chairs': [Point(128, 128).buffer(50) for _ in range(3)],
    'tables': [Point(64, 64).buffer(30) for _ in range(3)],
    'carpets': [Point(192, 192).buffer(40) for _ in range(3)],
    'plants': [Point(32, 32).buffer(20) for _ in range(3)],
    'walls': [Point(160, 160).buffer(60) for _ in range(3)]
    }
transform = RandomTransform()
dataset = ShapelyFeatureDataset(categories, transform=transform, subset_size=2)

# Get a sample
rasters, masks = dataset[0]
for i, (raster, mask) in enumerate(zip(rasters, masks)):
  print(f"Raster {i}:\n{raster}\nMask {i}:\n{mask}")

In [ ]:
from matplotlib import pyplot

# Example usage
features = [Point(128, 128).buffer(50) for _ in range(10)]  # Example Shapely features
transform = RandomTransform()
dataset = ShapelyFeatureDataset(features, transform=transform, subset_size=5)

# Get a sample
rasters, masks = dataset[0]

# Visualize the rasters
fig, axes = pyplot.subplots(1, len(rasters), figsize=(15, 5))
for i, raster in enumerate(rasters):
  axes[i].imshow(raster, cmap='gray')
  axes[i].set_title(f'Raster {i}')
  axes[i].axis('off')

pyplot.show()
